In [6]:
import PyPDF2
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import numpy as np

# Load the PDF and extract the text
pdf_file = open('/home/saksh-robocop/Documents/projects/PDF_Chat/ss983_midsem.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
num_pages = len(pdf_reader.pages)
text = ''
for page_num in range(num_pages):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()

# Preprocess the text
text = re.sub(r'[\n\r]+', ' ', text)  # Replace newlines with spaces
text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
text = text.lower()  # Convert to lowercase

# Load the pre-trained MISTRAL model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

def get_response(question, pdf_text):
    # Tokenize the question and PDF text
    inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True)
    pdf_inputs = tokenizer(pdf_text, return_tensors='pt', padding=True, truncation=True)

    # Generate the response
    outputs = model.generate(
        input_ids=inputs.input_ids,
        past_user_inputs=[pdf_inputs.input_ids.squeeze()],
        max_length=1024,
        do_sample=True,
        top_p=0.92,
        top_k=0,
        temperature=0.7,
        num_return_sequences=1,
    )

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

# Example usage
question = "What is the main topic of the PDF?"
print(get_response(question, text))

OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/mistralai/Mistral-7B-v0.1 and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.